In [ ]:
import polars as pl
import sys
import os

sys.path.append(os.path.dirname(os.path.abspath('')))

from libraries.client_stashapp import get_stashapp_client, StashAppClient

stash = get_stashapp_client()
stash_client = StashAppClient()

In [ ]:
all_tags = pl.DataFrame(stash.find_tags(fragment="id name"))

ai_tags = all_tags.filter(pl.col("name").str.ends_with("_AI"))
ai_tag_ids = ai_tags.select("id").to_series().to_list()
ai_tags

In [ ]:
stash_tags = stash_client.get_tags_by_names(["AI", "AI_Tagged", "Pussy Closeup_AI", "Video Cut"])

In [ ]:
for i in range(0, 296):
    video_cut_markers = stash.find_scene_markers(
        {
            "tags": {
                "value": [stash_tags.video_cut["id"]],
                "modifier": "INCLUDES_ALL",
            }
        },
        filter={ "per_page": 1000 },
        fragment="id primary_tag { id name } tags { id name } scene { id title }",
    )

    for video_cut_marker in video_cut_markers:
        stash.destroy_scene_marker(video_cut_marker["id"])

In [ ]:
# Get scenes that have both AI_Tagged and Pussy Closeup_AI tags
scenes_with_markers = stash.find_scenes({
    "tags": {
        "value": [stash_tags.ai_tagged["id"]],
        "modifier": "INCLUDES_ALL",
    },
}, { "sort": "id", "per_page": 1000, "page": 2 }, fragment="id title tags { id name parents { id name } } scene_markers { id primary_tag { id name } }")

# Convert to DataFrame and extract marker information
scenes_df = pl.DataFrame(scenes_with_markers)

# Create a DataFrame with all scene markers
markers_df = scenes_df.explode("scene_markers").with_columns([
    pl.col("scene_markers").struct.field("primary_tag").struct.field("id").alias("primary_tag_id"),
    pl.col("scene_markers").struct.field("primary_tag").struct.field("name").alias("primary_tag_name"),
])

# Create two separate dataframes - one for AI tags and one for all tags
ai_markers_df = markers_df.filter(pl.col("primary_tag_name").str.ends_with("_AI"))
all_markers_df = markers_df.filter(pl.col("primary_tag_id") != stash_tags.video_cut["id"])

# Group by scene and calculate unique AI tags
ai_tag_counts = (ai_markers_df
    .group_by(["id", "title", "tags"])
    .agg([
        pl.col("primary_tag_id").n_unique().alias("unique_ai_tags"),
        pl.col("primary_tag_name").alias("ai_tag_names")
    ])
    .with_columns([
        pl.col("ai_tag_names").list.unique().alias("ai_tag_names")
    ])
    .sort("unique_ai_tags")
)

# Group by scene and get all marker tags
all_tag_counts = (all_markers_df
    .group_by(["id", "title"])
    .agg([
        pl.col("primary_tag_id").n_unique().alias("unique_all_tags"),
        pl.col("primary_tag_name").alias("all_tag_names")
    ])
    .with_columns([
        pl.col("all_tag_names").list.unique().alias("all_tag_names")
    ])
)

# Join the results and format both columns
scenes_sort_by_marker_count = (ai_tag_counts
    .join(all_tag_counts, on=["id", "title"])
    .with_columns([
        pl.col("ai_tag_names").map_elements(lambda x: ", ".join(sorted(x)), return_dtype=pl.Utf8).alias("ai_tags_formatted"),
        pl.col("all_tag_names").map_elements(lambda x: ", ".join(sorted(x)), return_dtype=pl.Utf8).alias("all_marker_tags_formatted")
    ])
    .drop(["ai_tag_names", "all_tag_names"])
).sort("unique_ai_tags")
scenes_sort_by_marker_count


In [ ]:
scenes_to_be_reset = scenes_sort_by_marker_count.filter(
    pl.col("unique_ai_tags") == pl.col("unique_all_tags"),
    pl.col("unique_ai_tags") == 2
)
scenes_to_be_reset

In [ ]:
for selected_scene in scenes_to_be_reset.iter_rows(named=True):
    scene_id = selected_scene["id"]
    scene = stash.find_scene(scene_id, fragment="id title tags { id name parents { id name } } scene_markers { id primary_tag { id name parents { id name } } tags { id name } }")

    # Get all tags from the scene
    all_scene_tags = scene['tags']
    ai_tags = [tag for tag in all_scene_tags if any(parent['id'] == stash_tags.ai["id"] for parent in tag['parents'])]
    ai_tag_ids = [tag['id'] for tag in ai_tags]
    stash_client.bulk_scene_update([scene_id], ai_tag_ids + [stash_tags.ai_tagged["id"]], "REMOVE")

    scene_markers = scene['scene_markers']
    for marker in scene_markers:
        # IF any of the parents of the primary tag is AI, then remove the marker
        if any(parent['id'] == stash_tags.ai["id"] for parent in marker['primary_tag']['parents']):
            stash.destroy_scene_marker(marker["id"])